In [1]:
import time
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM.joining.sim import SIM
from fastfusion.mapper.FFM.joining.simexplore import join_sims
import fastfusion.mapper.FFM.exploration.mapper_one_einsum as mapper_one_einsum

from fastfusion.mapper.FFM.exploration.mapping_filter_tags.ffmt import get_ffmt_tag
from fastfusion.mapper.FFM.exploration.mapping_filter_tags.onesplit import get_one_split_tag
from fastfusion.mapper.FFM.pareto import PartialMappings


def get_fused_mappings(spec: Specification, tagger=None) -> PartialMappings:
    spec.estimate_energy_area()
    flattened_architecture = spec.get_flattened_architecture()
    t0 = time.time()
    sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger)
    pmapping_time = time.time() - t0
    total_pmappings = sum(p.mappings.n_pmappings for v in sims.values() for p in v)
    print(f'Took {pmapping_time:.2f} seconds to generate {total_pmappings} partial mappings ({total_pmappings / pmapping_time:.2f} per second)')
    t0 = time.time()
    mappings = join_sims(sims, spec, flattened_architecture, drop_valid_reservations=archname != "snowcat")
    join_time = time.time() - t0
    mappings.decompress(decompress_data)
    print(f"Pmappings: {pmapping_time:.2f}. Joining: {join_time:.2f}. Total Pmappings: {total_pmappings}. Total mappings: {mappings.n_pmappings}. Time per pmapping: {pmapping_time / total_pmappings:.2e}")
    return mappings

# archname = "snowcat"
archname = "four_level"
spec = Specification.from_yaml(
    f"architecture/{archname}.arch.yaml",
    "workloads/mha_full.workload.yaml",
    "workloads/mha_full.renames.yaml"
)

result = {
    # "LoopTree": get_fused_mappings(spec, get_one_split_tag).data,
    "LoopForest": get_fused_mappings(spec).data,
    # "FFMT": get_fused_mappings(spec, get_ffmt_tag).data,
}
# Pmappings: 65.17. Joining: 44.25. Total Pmappings: 2186997.5. Total mappings: 1.5343799278736949e+29. Time per pmapping: 2.98e-05

INFO        Loading yaml file architecture/four_level.arch.yaml
INFO        Found top key variables in architecture/four_level.arch.yaml
INFO        Found top key architecture in architecture/four_level.arch.yaml
INFO        Found top key component_classes in architecture/four_level.arch.yaml
INFO        Loading yaml file workloads/mha_full.workload.yaml
INFO        Found top key workload in workloads/mha_full.workload.yaml
INFO        Loading yaml file workloads/mha_full.renames.yaml
INFO        Found top key renames in workloads/mha_full.renames.yaml
WARNING     Loading configuration file from /root/.config/fastfusion/config.yaml
INFO        Calculated "1024*1024*128*8" = 1073741824.
INFO        Calculated "1024*1024*32*8" = 268435456.
INFO        Calculated "0.5" = 0.5.
Generating storage and loop choices for Einsum I: 6it [00:00, 306.92it/s]
Generating storage and loop choices for Einsum V: 51it [00:00, 404.85it/s]
Generating storage and loop choices for Einsum K: 51it [00:00, 473.

Took 40.07 seconds to generate 2186997.5 partial mappings (54581.88 per second)
SIM I tensors: {'I'}
SIM V tensors: {'V', 'I'}
SIM K tensors: {'K', 'I'}
SIM Q tensors: {'Q', 'I'}
SIM QK tensors: {'QK', 'K', 'Q'}
SIM AV tensors: {'V', 'QK', 'AV'}
SIM Z tensors: {'Z', 'AV'}
SIM FFA tensors: {'FFA', 'Z'}
SIM FFB tensors: {'FFA'}


Grouping Partial Mappings: 100%|██████████| 69/69 [00:00<00:00, 1863.82it/s]


Initial consolidate and group: 42.55 seconds

Einsum V (2/9)
Consolidating: 0.00 seconds


Grouping Partial Mappings: 100%|██████████| 38/38 [00:00<00:00, 27432.63it/s]


...

Combining: 0.16 seconds
Grouping: 0.00 seconds
Bucket merging: 0.01 seconds


Merging mappings FFA <--> FFB: 100%|██████████| 69/69 [00:00<00:00, 1643.54it/s]


Mapping merging: 0.14 seconds
Scaled runtime by 1.0. Runtime: 1.40
	Combining 123(69) x 123(69) -> 69
	Number of groups for Einsum FFB: 69
	Number of mappings for Einsum FFB: 138
	Mappings per group for Einsum FFB: 2.0


Grouping Partial Mappings: 100%|██████████| 1/1 [00:00<00:00, 1473.75it/s]



======== Total time ========
Initial consolidate and group: 42.55 seconds
Consolidating: 0.11 seconds
Combining: 0.65 seconds
Grouping: 0.00 seconds
Bucket merging: 0.26 seconds
Removing mappings that can't be combined later: 0.10 seconds
Mapping merging: 3.10 seconds

Total: 46.77 seconds

Pmappings: 40.07. Joining: 47.13. Total Pmappings: 2186997.5. Total mappings: 1.5343799278736949e+29. Time per pmapping: 1.83e-05


In [2]:
import importlib
import fastfusion.visualization.interactive
importlib.reload(fastfusion.visualization.interactive)
from fastfusion.visualization.interactive import plotly_show
# mappings2.make_pareto(columns=["RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy"])
# plotly_show(result, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", category="Category", logscales=False, einsum_names=spec.workload.einsum_names)
plotly_show(result, "metric_Latency", "metric_Energy", logscales=False, einsum_names=spec.workload.einsum_names)
# from fastfusion.visualization.interactive import mapping2svg
# mapping2svg(mappings.data.iloc[0], spec.workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [3]:
mappings.data.sort_values(by="metric_Energy", ascending=True).head()
from fastfusion.mapper.FFM.visualization import make_mapping
from IPython.display import SVG
newmapping = make_mapping(mappings.data.iloc[0], spec.workload.einsum_names)
display(SVG(newmapping.render()))

# {'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1
# TODO: Re-add -1 to the mapper one eisnum freenig
# compatibility2sims['Matmul1']["{'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1"]
# Above 1: 8192
# Above 2: 8321
# compatibility2sims['Matmul2']["{'n1'}-1 || [GlobalBuffer] T1 sz 0 above 1, [GlobalBuffer] T2 sz 0 above 0"]

NameError: name 'mappings' is not defined

In [ ]:
compatibility2sims = {einsum_name: {s.compatibility_str(): s for s in sims2} for einsum_name, sims2 in sims.items()}
print(compatibility2sims)